# Mission statement #

Want to build a neural network predicting the outcome of a match, given the hero matchup (5 radiant heroes and 5 dire)

In [1]:
import requests
import pprint
import json
import pandas as pd
import numpy as np

Get the hero dictionary:

In [2]:
h = requests.get("https://api.opendota.com/api/heroes")
heroes = json.loads(h.text)
hero_df = pd.DataFrame(heroes)

# Mine the required data. #

We want to get high-mmr matches, using OpenDota API which gives a random sample of 100 recent games. Work iteratively - per every call, sort out the 100 matches, get 100 older matches and repeat.

In [76]:
def pickaxe(below_match_id = None):
    
    # Pick out relevant matches from a sample of 100
    
    if below_match_id == None:
        string = 'https://api.opendota.com/api/publicMatches'
    else:
        string = 'https://api.opendota.com/api/publicMatches?less_than_match_id=' + str(below_match_id)
    a = requests.get(string)
    A = json.loads(a.text)
    over4k = []
    over5k = []
    for i in range(len(A)):
        if A[i]['avg_mmr'] is not None:
            if A[i]['game_mode'] == 22 and A[i]['avg_mmr'] > 4000 and A[i]['avg_mmr'] < 5000:
                over4k.append(A[i])
            elif A[i]['game_mode'] == 2 and A[i]['avg_mmr'] > 5000:
                over5k.append(A[i]) 
    lowest_match_id = A[-1]['match_id']
    return over4k, over5k, lowest_match_id

def picked_heroes(hero_string):
    
    # Convert the string from pickaxe() to a length 116 array with 1s on picked heroes, 0 elsewhere.
    
    team_input = np.zeros(len(hero_df))
    y = [int(i) for i in hero_string.split(',')]
    for item in y:
        idx = int(hero_df[hero_df['id'] == item].index[0])
        team_input[idx] += 1
    return team_input

def process_the_pickaxe(game_list):
    Xt = np.zeros(232)
    yt = np.array(0)
    for item in game_list:
        X = np.hstack((picked_heroes(item['radiant_team']), picked_heroes(item['dire_team'])))
        y = np.array(0)
        y += item['radiant_win']*1
        Xt = np.vstack((Xt,X))
        yt = np.vstack((yt,y))
    Xt = np.delete(Xt, 0, 0)
    yt = np.delete(yt, 0, 0)
    if Xt.shape == (231,):
        Xt = None
        yt = None
    return Xt, yt

def single_call(last_match_id = None):
    
    # Perform a call for 100 matches and process it.
    
    if last_match_id == None:
        a,b,c = pickaxe()
    else:
        a,b,c = pickaxe(last_match_id)
    X_4, y_4 = process_the_pickaxe(a)
    X_5, y_5 = process_the_pickaxe(b)
    return X_4, y_4, X_5, y_5, c

def multiple_calls(call_number = 60, last_match_id = None):
    Total_X_4 = np.zeros(232)
    Total_X_5 = np.zeros(232)
    Total_y_4 = np.array(0)
    Total_y_5 = np.array(0)
    for i in range(call_number):
        X_4, y_4, X_5, y_5, last_match_id = single_call(last_match_id = last_match_id)
        if X_4 is not None:
            Total_X_4 = np.vstack((Total_X_4, X_4))
            Total_y_4 = np.vstack((Total_y_4, y_4))
        if X_5 is not None:
            Total_X_5 = np.vstack((Total_X_5, X_5))
            Total_y_5 = np.vstack((Total_y_5, y_5))
            continue
    Total_X_4 = np.delete(Total_X_4, 0, 0)
    Total_X_5 = np.delete(Total_X_5, 0, 0)
    Total_y_4 = np.delete(Total_y_4, 0, 0)
    Total_y_5 = np.delete(Total_y_5, 0, 0)
    return Total_X_4, Total_y_4, Total_X_5, Total_y_5, last_match_id

#### Playground:####

In [74]:
X4, y4, X5, y5, _ = multiple_calls()

c:\users\gintas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: in the future the special handling of scalars will be removed from delete and raise an error
c:\users\gintas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:78: DeprecationWarning: in the future the special handling of scalars will be removed from delete and raise an error


In [75]:
print(X4.shape, y4.shape, X5.shape, y5.shape)

(524, 232) (524, 1) (231,) ()


In [28]:
Xt = np.zeros(232)
Xt = np.delete(Xt, 0, 0)
Xt.shape

(231,)

In [77]:
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

In [78]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'